In [13]:
# 01_load_data.py
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Đường dẫn lưu dữ liệu
data_dir = 'D:/CV/BTL02/Train' # Thư mục chứa GTSRB Train đã giải nén
output_dir = 'processed_data'  # Thư mục lưu dữ liệu đã xử lý

# Hàm tải dữ liệu GTSRB
def load_gtsrb(data_dir):
    """Hàm tải dữ liệu GTSRB từ thư mục"""
    images = []  # Danh sách chứa ảnh
    labels = []  # Danh sách chứa nhãn
    print("Đang tải dữ liệu GTSRB từ:", data_dir)
    for label in range(43):  # 43 lớp biển báo
        class_dir = os.path.join(data_dir, str(label))
        if not os.path.exists(class_dir):
            continue
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            try:
                img = load_img(img_path, target_size=(32, 32))  # Resize ảnh về 32x32
                img = img_to_array(img) / 255.0  # Chuẩn hóa giá trị pixel về [0, 1]
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Lỗi tải ảnh {img_path}: {e}")
    return np.array(images), np.array(labels)

# Kiểm tra dữ liệu
if not os.path.exists(data_dir):
    print("Dữ liệu GTSRB chưa có. Vui lòng tải từ Kaggle!")
    print("1. Tải file 'gtsrb-german-traffic-sign.zip' từ: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")
    print("2. Giải nén vào thư mục 'gtsrb_data'")
    print("3. Chạy lại file này sau khi giải nén.")
else:
    X, y = load_gtsrb(data_dir)  # Tải dữ liệu
    y = tf.keras.utils.to_categorical(y, 43)  # Chuyển nhãn thành one-hot encoding

    # Chia dữ liệu: 70% train, 20% val, 10% test
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)
    
    # Tạo thư mục lưu dữ liệu đã xử lý
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Lưu dữ liệu
    np.save(os.path.join(output_dir, 'X_train.npy'), X_train)
    np.save(os.path.join(output_dir, 'y_train.npy'), y_train)
    np.save(os.path.join(output_dir, 'X_val.npy'), X_val)
    np.save(os.path.join(output_dir, 'y_val.npy'), y_val)
    np.save(os.path.join(output_dir, 'X_test.npy'), X_test)
    np.save(os.path.join(output_dir, 'y_test.npy'), y_test)
    print(f"Dữ liệu đã được lưu vào: {output_dir}")
    print(f"Kích thước: train={X_train.shape}, val={X_val.shape}, test={X_test.shape}")

Đang tải dữ liệu GTSRB từ: D:/CV/BTL02/Train
Dữ liệu đã được lưu vào: processed_data
Kích thước: train=(27446, 32, 32, 3), val=(7881, 32, 32, 3), test=(3882, 32, 32, 3)


In [ ]:
# 02_train_model.py
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import os

# Kiểm tra GPU
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.test.is_gpu_available())
print("Devices:", tf.config.list_physical_devices())

# Đường dẫn
data_dir = 'processed_data'  # Thư mục chứa dữ liệu đã xử lý
model_path = 'traffic_sign_cnn01.h5'  # Đường dẫn lưu mô hình

# Tải dữ liệu
if not os.path.exists(data_dir):
    print("Dữ liệu chưa được xử lý! Chạy file '01_load_data.py' trước.")
    exit()

X_train = np.load(os.path.join(data_dir, 'X_train.npy'))
y_train = np.load(os.path.join(data_dir, 'y_train.npy'))
X_val = np.load(os.path.join(data_dir, 'X_val.npy'))
y_val = np.load(os.path.join(data_dir, 'y_val.npy'))
X_test = np.load(os.path.join(data_dir, 'X_test.npy'))
y_test = np.load(os.path.join(data_dir, 'y_test.npy'))
print("Đã tải dữ liệu từ file .npy")

# Xây dựng mô hình CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'),  # Lớp tích chập 1
    BatchNormalization(),  # Chuẩn hóa batch để tăng tốc huấn luyện
    MaxPooling2D(pool_size=(2, 2)),  # Giảm kích thước không gian
    Conv2D(64, (3, 3), activation='relu', padding='same'),  # Lớp tích chập 2
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),  # Lớp tích chập 3
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),  # Chuyển thành vector
    Dense(256, activation='relu'),  # Lớp fully connected
    Dropout(0.5),  # Dropout để tránh overfitting
    Dense(43, activation='softmax')  # Lớp đầu ra với 43 nhãn
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()  # Hiển thị cấu trúc mô hình

# Huấn luyện mô hình
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)  # Dừng sớm nếu val_loss không cải thiện
history = model.fit(X_train, y_train, 
                    epochs=30,  # Số lần huấn luyện
                    batch_size=64,  # Kích thước batch
                    validation_data=(X_val, y_val),  # Dữ liệu kiểm tra
                    shuffle=True,  # Xáo trộn dữ liệu
                    callbacks=[early_stopping],
                    verbose=1)  # Hiển thị tiến trình

# Đánh giá mô hình
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Độ chính xác trên tập test: {test_accuracy*100:.2f}%")

# Lưu mô hình
model.save(model_path)
print(f"Mô hình đã được lưu vào: {model_path}")

TensorFlow version: 2.18.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU available: False
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Đã tải dữ liệu từ file .npy


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

...

 Total params: 629,739 (2.40 MB)

 Trainable params: 629,291 (2.40 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/30
429/429 ━━━━━━━━━━━━━━━━━━━━ 31s 68ms/step - accuracy: 0.3742 - loss: 2.4576 - val_accuracy: 0.7720 - val_loss: 0.8013
Epoch 2/30
429/429 ━━━━━━━━━━━━━━━━━━━━ 29s 68ms/step - accuracy: 0.8999 - loss: 0.3111 - val_accuracy: 0.9758 - val_loss: 0.0871
Epoch 3/30
429/429 ━━━━━━━━━━━━━━━━━━━━ 34s 80ms/step - accuracy: 0.9611 - loss: 0.1243 - val_accuracy: 0.9780 - val_loss: 0.0689
Epoch 4/30
429/429 ━━━━━━━━━━━━━━━━━━━━ 33s 78ms/step - accuracy: 0.9724 - loss: 0.0863 - val_accuracy: 0.9786 - val_loss: 0.0661
Epoch 5/30
429/429 ━━━━━━━━━━━━━━━━━━━━ 32s 75ms/step - accuracy: 0.9803 - loss: 0.0607 - val_accuracy: 0.9895 - val_loss: 0.0325
Epoch 6/30
429/429 ━━━━━━━━━━━━━━━━━━━━ 34s 79ms/step - accuracy: 0.9854 - loss: 0.0437 - val_accuracy: 0.9636 - val_loss: 0.1293
Epoch 7/30
429/429 ━━━━━━━━━━━━━━━━━━━━ 33s 78ms/step - accuracy: 0.9837 - loss: 0.0503 - val_accuracy: 0.9853 - val_loss: 0.0496
Epoch 8/30
429/429 ━━━━━━━━━━━━━━━━━━━━ 31s 72ms/step - accuracy: 0.9845 - loss: 0.0504 - 

Độ chính xác trên tập test: 98.74%
Mô hình đã được lưu vào: traffic_sign_model.h5
